In [ ]:
import matplotlib.pyplot as plt 
import matplotlib
import cartopy
import cartopy.crs as ccrs
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import scipy

%run ./Tracking_Functions.py

# Load and prepare the ERA5 data (IVT, IVT reference climatology, sea level pressure)

In [6]:
# ds_ivt=xr.open_dataset('/work/aa0049/a271109/data_misc/AR_Guan/code/input_ivt/2020/ivt_202004.nc')
ds_ivt=xr.open_dataset('/work/aa0049/a271109/data_misc/AR_Guan/code/input_ivt/2019/ivt_201904.nc')

Time=np.array(pd.DataFrame(ds_ivt.time.values,columns=['Datetime'])['Datetime'].values.astype('datetime64[s]').tolist()) #[:30]
Time_sel=Time

ds_ivt=ds_ivt.sel(time=Time_sel) 

Lon, Lat= xr.broadcast(ds_ivt.lon, ds_ivt.lat)[0].values.T, xr.broadcast(ds_ivt.lon, ds_ivt.lat)[1].values.T[:,:]
Mask=1*(Lat>-999) 
ivte = ds_ivt.ivtx.isel(ens=0,lev=0).values 
ivtn = ds_ivt.ivty.isel(ens=0,lev=0).values


In [8]:
ds_era_msl=xr.open_mfdataset('/work/aa0049/a271109/data_misc/ERA5/data_processed/MSL_ERA5_2019-04_1H_remapped.nc')

In [ ]:
ds_ivt_pctl=xr.open_dataset('/work/aa0049/a271109/data_misc/AR_Guan/code/ivt_percentile_mlauer.nc',decode_times=False)
IVTtrheshold=ds_ivt_pctl.ivt1.isel(ens=0,lev=0).values 

# Run MOAAP

In [10]:
moaap(Lon = Lon,                           # 2D longitude grid centers
      Lat = Lat,                           # 2D latitude grid spacing
      Time = Time_sel,                         # datetime vector of data
      dT = 1,                              # integer - temporal frequency of data [hour]
      Mask = Mask,                         # mask with dimensions [lat,lon] defining analysis region
      
      ivte = ivte, # zonal integrated vapor transport [kg m-1 s-1]
      ivtn = ivtn, # meidional integrated vapor transport [kg m-1 s-1]
      
      IVTtrheshold = IVTtrheshold, # Integrated water vapor transport threshold for AR detection [kg m-1 s-1]
      # JLa: additionall fixed threshold 100 in code
      MinTimeIVT=9,  # minimum livetime of ARs [hours]
      AR_MinLen=1500,  # mimimum length of an AR [km]
      AR_Lat=0,  # AR centroids have to be poeward of this latitude
      AR_width_lenght_ratio=2,  # mimimum length to width ratio of AR

      slp=ds_era_msl.MSL.values,  # sea level pressure [Pa]
      # cyclone tracking
      MinTimeCY=12,  # minimum livetime of cyclones [hours]
      MaxPresAnCY=-8,  # preshure thershold for cyclone anomaly [hPa]
      # anty cyclone tracking
      MinTimeACY=12,  # minimum livetime of anticyclone [hours]
      MinPresAnACY=6,  # preshure thershold for anti cyclone anomaly [hPa]
      
      DataName = 'ERA5_2019-04_global', #'ERA5_2020-04_global',
      OutputFolder='./output/'
     )

 
The provided variables allow tracking the following phenomena
 
|  phenomenon  | tracking |
---------------------------
   Jetstream   |   no
   PSL CY/ACY  |   yes
   Z500 CY/ACY |   no
   COLs        |   no
   IVT ARs     |   yes
   MS ARs      |   no
   Fronts      |   no
   TCs         |   no
   MCSs        |   no
   Equ. Waves  |   no
---------------------------
 
======> track IVT streams and atmospheric rivers (ARs)
        10342 object found
        break up long living IVT objects that have many elements


100%|██████████| 1022/1022 [06:01<00:00,  2.83it/s] 


        00:06:25.12
9
            Loop over 2513 objects
        check if MSs quallify as ARs
            Loop over 2512 objects


/work/aa0049/a271109/tools/MOAAP_AWI/Tracking_Functions.py:255: RuntimeWarning: All-NaN slice encountered
  obj_min = np.nanmin(data_slice, axis=(1, 2))
/work/aa0049/a271109/tools/MOAAP_AWI/Tracking_Functions.py:256: RuntimeWarning: All-NaN slice encountered
  obj_max = np.nanmax(data_slice, axis=(1, 2))
/work/aa0049/a271109/tools/MOAAP_AWI/Tracking_Functions.py:257: RuntimeWarning: Mean of empty slice
  obj_mean = np.nanmean(data_slice, axis=(1, 2))
/sw/spack-levante/mambaforge-22.9.0-2-Linux-x86_64-kptncg/lib/python3.10/site-packages/scipy/ndimage/_measurements.py:1407: RuntimeWarning: invalid value encountered in double_scalars
  results = [sum(input * grids[dir].astype(float), labels, index) / normalizer


        00:09:24.84
======> track cyclones from PSL
        track cyclones
            915 object found
            break up long living CY objects that heve many elements


100%|██████████| 16/16 [02:14<00:00,  8.38s/it]


        00:02:39.24
        track anti-cyclones
        1870 object found
            break up long living ACY objects that have many elements


100%|██████████| 16/16 [02:27<00:00,  9.23s/it]


        00:02:53.53
            Loop over 183 objects
            Loop over 75 objects
        00:07:19.74
 
Save the object masks into a joint netCDF
Saved: ./output/ObjectMasks_ERA5_2019-04_global.nc
        00:03:11.30
